<a href="https://colab.research.google.com/github/igornunespatricio/house-prices-advanced-regression-techniques/blob/main/pipeline_gbr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages

In [ ]:
# run this if you are in google colab
# !pip install opendatasets >/dev/null
# !pip install --upgrade scikit-learn >/dev/null

# Importing packages

In [ ]:
import opendatasets
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
import joblib

# Downloading data

In [ ]:
# Before running this code, make sure to upload your kaggle.json file here, otherwise it will return an error.
opendatasets.download('https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques')

Skipping, found downloaded files in "./house-prices-advanced-regression-techniques" (use force=True to force download)


# Reading data

In [ ]:
# Reading the database
train = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('house-prices-advanced-regression-techniques/test.csv')

# Data Analysis

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

Many columns have null values, we know from the data description that some of them are null because they don't have the feature, for example, garage or basement, so they shouldn't be null. In this case, we can change these values to something more meaningful than just removing the rows/columns or adding the mean, median or mode.

In [ ]:
train.isnull().sum().sort_values(ascending=False).head(30)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
Electrical         1
Id                 0
Functional         0
Fireplaces         0
KitchenQual        0
KitchenAbvGr       0
BedroomAbvGr       0
HalfBath           0
FullBath           0
BsmtHalfBath       0
TotRmsAbvGrd       0
GarageCars         0
dtype: int64

In [ ]:
x = train.drop(['SalePrice', 'Id'], axis=1)
y = train['SalePrice']

In [ ]:
# printing all columns to be passed to the model
print(*x.columns.tolist(), sep='\n')

MSSubClass
MSZoning
LotFrontage
LotArea
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
MasVnrArea
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinSF1
BsmtFinType2
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
Heating
HeatingQC
CentralAir
Electrical
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
KitchenQual
TotRmsAbvGrd
Functional
Fireplaces
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars
GarageArea
GarageQual
GarageCond
PavedDrive
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
PoolQC
Fence
MiscFeature
MiscVal
MoSold
YrSold
SaleType
SaleCondition


In [ ]:
x['LotFrontage'].isnull().value_counts(dropna=False)

LotFrontage
False    1201
True      259
Name: count, dtype: int64

For the following columns we should input a "No Feature" value since as per the data description, they don't contain the feature, for example, don't contain garage, pool, basement, etc.

```
'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu',
'GarageYrBlt', 'GarageCond', 'GarageType', 'GarageFinish', 'GarageQual',
'BsmtFinType2', 'BsmtExposure', 'BsmtQual', 'BsmtCond', 'BsmtFinType1'
```

# Pipeline

In [ ]:
class GroupImputer(BaseEstimator, TransformerMixin):
    def __init__(self, agg_func='median'):
        self.agg_func = agg_func

    def fit(self, x, y=None):
        self.group_columns = x.select_dtypes(include='object').columns.tolist()
        self.value_columns = x.select_dtypes(include=np.number).columns.tolist()
        self.aggs = x.groupby(self.group_columns)
        return self

    def transform(self, x):
        x_transformed = x.copy()
        column = self.aggs[self.value_columns].transform(self.agg_func)
        x_transformed[self.value_columns] = x[self.value_columns].fillna(column)
        return x_transformed

    def fit_transform(self, x, y=None):
        self.fit(x)
        result = self.transform(x)
        return result

    def get_feature_names_out(self, input_features=None):
        return input_features

# building the scorer function
def rmse_of_log(y_true, y_pred):
    """
    This scorer function will calculate the RMSE of the log of predictions and true values
    """
    return root_mean_squared_error(np.log1p(y_true), np.log1p(y_pred))

In [ ]:
train.select_dtypes(include='object').columns.tolist()

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [ ]:
random_state = 42

cat_ohe_cols = [
    'MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', #'Neighborhood'    not sure i wanna use neighborhood, to many values
    'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
    'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
    'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition'
]

cat_oe_cols = [
    'LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond',
    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
    'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'FireplaceQu',
    'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive',
    'PoolQC', 'Fence'
]

cat_oe_values = [
    ['IR3', 'IR2', 'IR1', 'Regular'],  # LotShape
    ['ELO', 'NoSeWa', 'NoSewr', 'AllPub'],  # Utilities
    ['Sev', 'Mod', 'Gtl'],  # LandSlope
    ['Po', 'Fa', 'TA', 'Gd', 'Ex'],  # ExterQual
    ['Po', 'Fa', 'TA', 'Gd', 'Ex'],  # ExterCond
    ['No Feature', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],  # BsmtQual
    ['No Feature', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],  # BsmtCond
    ['No Feature', 'No', 'Mn', 'Av', 'Gd'],  # BsmtExposure
    ['No Feature', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],  # BsmtFinType1
    ['No Feature', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],  # BsmtFinType2
    ['Po', 'Fa', 'TA', 'Gd', 'Ex'],  # HeatingQc
    ['No', 'Yes'],  # CentralAir
    ['Po', 'Fa', 'TA', 'Gd', 'Ex'],  # KitchenQual
    ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],  # Functional
    ['No Feature', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],  # FireplaceQu
    ['No Feature', 'Unf', 'RFn', 'Fin'],  # GarageFinish
    ['No Feature', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],  # GarageQual
    ['No Feature', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],  # GarageCond
    ['N', 'P', 'Y'],  # PavedDrive
    ['No Feature', 'Fa', 'TA', 'Gd', 'Ex'],  #PoolQC
    ['No Feature', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv'],  # Fence
]



# cat_cols = x.select_dtypes(include='object').columns.tolist()
num_cols = x.select_dtypes(include='number').columns.tolist()[1:]

ohe_pipeline = Pipeline(
    steps=[
        ('categorical_imputer_no_feature_ohe', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='No Feature')),
        ('one_hot_encoder', OneHotEncoder(handle_unknown='infrequent_if_exist', min_frequency=0.2, sparse_output=False))

    ]
)

oe_pipeline = Pipeline(
    steps=[
        ('categorical_imputer_no_feature_oe', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='No Feature')),
        ('ordinal_encoder', OrdinalEncoder(categories=cat_oe_values, handle_unknown='use_encoded_value', unknown_value=-1, encoded_missing_value=-1, min_frequency=0.2))
    ]
)

categorical_preprocessor = ColumnTransformer(
    transformers=[
        ('cat', ohe_pipeline, cat_ohe_cols),
        ('oe', oe_pipeline, cat_oe_cols)
    ],
    sparse_threshold=0
)

numerical_pipeline = Pipeline(
    steps=[
        ('numerical_imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
        ('scaler', StandardScaler())
    ]
)

numerical_preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, num_cols)
    ],
    sparse_threshold=0
)

feature_union = FeatureUnion(
    transformer_list=[
        ('categorical_preprocessor', categorical_preprocessor),
        ('numerical_preprocessor', numerical_preprocessor)
    ]
)

full_pipeline = Pipeline(
    steps=[
        ('feature_union', feature_union),
        ('pca', PCA(n_components=30, random_state=random_state)),
        ('regressor', GradientBoostingRegressor(random_state=random_state))
    ]
)

full_pipeline

param_grid = {
    'pca__n_components': [50, 80, 100],
    'regressor__n_estimators': [250, 300, 350],
    'regressor__max_depth': [2, 5, 8]
}

rmse_log_scorer= make_scorer(rmse_of_log, greater_is_better=False)

grid_search = GridSearchCV(full_pipeline, param_grid, cv=5, scoring=rmse_log_scorer, verbose=4)

grid_search.fit(x, y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END pca__n_components=10, regressor__max_depth=2, regressor__n_estimators=250;, score=-0.144 total time=   2.0s
[CV 2/5] END pca__n_components=10, regressor__max_depth=2, regressor__n_estimators=250;, score=-0.165 total time=   1.3s
[CV 3/5] END pca__n_components=10, regressor__max_depth=2, regressor__n_estimators=250;, score=-0.153 total time=   1.3s
[CV 4/5] END pca__n_components=10, regressor__max_depth=2, regressor__n_estimators=250;, score=-0.144 total time=   1.3s
[CV 5/5] END pca__n_components=10, regressor__max_depth=2, regressor__n_estimators=250;, score=-0.139 total time=   1.3s
[CV 1/5] END pca__n_components=10, regressor__max_depth=2, regressor__n_estimators=300;, score=-0.145 total time=   1.5s
[CV 2/5] END pca__n_components=10, regressor__max_depth=2, regressor__n_estimators=300;, score=-0.164 total time=   1.6s
[CV 3/5] END pca__n_components=10, regressor__max_depth=2, regressor__n_estimators=300;, sc

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_union',
                                        FeatureUnion(transformer_list=[('categorical_preprocessor',
                                                                        ColumnTransformer(sparse_threshold=0,
                                                                                          transformers=[('cat',
                                                                                                         Pipeline(steps=[('categorical_imputer_no_feature_ohe',
                                                                                                                          SimpleImputer(fill_value='No '
                                                                                                                                                   'Feature',
                                                                                                                                        strategy='constant')),
                                                                                                                         ('one_hot_encoder',
                                                                                                                          OneHotEncoder(handle_unknown='infreque...
                                                                                                          'EnclosedPorch',
                                                                                                          '3SsnPorch', ...])]))])),
                                       ('pca',
                                        PCA(n_components=30, random_state=42)),
                                       ('regressor',
                                        GradientBoostingRegressor(random_state=42))]),
             param_grid={'pca__n_components': [10, 30, 50],
                         'regressor__max_depth': [2, 5, 8],
                         'regressor__n_estimators': [250, 300, 350]},
             scoring=make_scorer(rmse_of_log, greater_is_better=False, response_method='predict'),
             verbose=4)

In [ ]:
pd.DataFrame(grid_search.cv_results_).sort_values(by='rank_test_score', ascending=True).head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_pca__n_components,param_regressor__max_depth,param_regressor__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
22,13.797321,0.093872,0.028051,0.006394,50,5,300,"{'pca__n_components': 50, 'regressor__max_dept...",-0.130455,-0.144058,-0.141783,-0.126539,-0.153002,-0.139167,0.009566,1
23,15.951511,0.156335,0.026603,0.007332,50,5,350,"{'pca__n_components': 50, 'regressor__max_dept...",-0.130506,-0.144175,-0.141685,-0.126473,-0.153055,-0.139179,0.009596,2
21,11.415235,0.111776,0.024893,0.004013,50,5,250,"{'pca__n_components': 50, 'regressor__max_dept...",-0.130551,-0.144040,-0.141886,-0.126711,-0.153095,-0.139257,0.009534,3
19,6.040935,0.160820,0.024759,0.006685,50,2,300,"{'pca__n_components': 50, 'regressor__max_dept...",-0.136811,-0.155766,-0.144202,-0.135432,-0.138449,-0.142132,0.007443,4
12,7.076231,0.235812,0.025259,0.005658,30,5,250,"{'pca__n_components': 30, 'regressor__max_dept...",-0.137170,-0.149015,-0.139525,-0.135575,-0.149397,-0.142136,0.005909,5
13,8.503559,0.240600,0.024600,0.003591,30,5,300,"{'pca__n_components': 30, 'regressor__max_dept...",-0.137131,-0.148854,-0.139453,-0.135666,-0.149595,-0.142140,0.005914,6
14,9.731916,0.239930,0.027196,0.007216,30,5,350,"{'pca__n_components': 30, 'regressor__max_dept...",-0.137266,-0.148818,-0.139540,-0.135531,-0.149591,-0.142149,0.005904,7
20,7.088191,0.260589,0.031878,0.008470,50,2,350,"{'pca__n_components': 50, 'regressor__max_dept...",-0.135796,-0.157284,-0.144205,-0.135764,-0.138532,-0.142316,0.008092,8
18,5.074415,0.239177,0.023425,0.004884,50,2,250,"{'pca__n_components': 50, 'regressor__max_dept...",-0.137823,-0.154863,-0.143910,-0.135949,-0.139362,-0.142381,0.006774,9
9,3.178051,0.247644,0.020198,0.000616,30,2,250,"{'pca__n_components': 30, 'regressor__max_dept...",-0.140034,-0.173518,-0.140538,-0.138234,-0.142665,-0.146998,0.013335,10


In [ ]:
grid_search.best_params_

{'pca__n_components': 50,
 'regressor__max_depth': 5,
 'regressor__n_estimators': 300}

Looking at the hyperparameters, there is still room for improvment since the best choice for n_components from PCA was 50 the top value from the list. We can increase the number components and train again.

In [ ]:
joblib.dump(grid_search, 'pipeline_gbr.pkl')

['pipeline_gbr.pkl']

# Submitting result to Kaggle

In [ ]:
model_saved = joblib.load('/content/pipeline_gbr.pkl')

In [28]:
model_saved

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_union',
                                        FeatureUnion(transformer_list=[('categorical_preprocessor',
                                                                        ColumnTransformer(sparse_threshold=0,
                                                                                          transformers=[('cat',
                                                                                                         Pipeline(steps=[('categorical_imputer_no_feature_ohe',
                                                                                                                          SimpleImputer(fill_value='No '
                                                                                                                                                   'Feature',
                                                                                                                                        strategy='constant')),
                                                                                                                         ('one_hot_encoder',
                                                                                                                          OneHotEncoder(handle_unknown='infreque...
                                                                                                          'EnclosedPorch',
                                                                                                          '3SsnPorch', ...])]))])),
                                       ('pca',
                                        PCA(n_components=30, random_state=42)),
                                       ('regressor',
                                        GradientBoostingRegressor(random_state=42))]),
             param_grid={'pca__n_components': [10, 30, 50],
                         'regressor__max_depth': [2, 5, 8],
                         'regressor__n_estimators': [250, 300, 350]},
             scoring=make_scorer(rmse_of_log, greater_is_better=False, response_method='predict'),
             verbose=4)

In [ ]:
predictions = model_saved.predict(test.drop('Id', axis=1))

In [ ]:
test_predictions = pd.DataFrame(
    {
        'Id': test.Id,
        'SalePrice': predictions
    }
)

test_predictions. head(5)

,Id,SalePrice
0,1461,128982.278412
1,1462,163259.622628
2,1463,184425.504053
3,1464,187342.210872
4,1465,175428.710329


In [ ]:
test_predictions.to_csv('pipeline_gradient_boosting_regressor_predictions.csv', index=False)

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f /content/pipeline_gradient_boosting_regressor_predictions.csv -m "Gradient Boosting Regressor with Pipeline, One Hot Encoder, Ordinal Encoder and 50 components in PCA"

100% 33.7k/33.7k [00:00<00:00, 54.2kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques